# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.automl.runtime.onnx_convert import OnnxConverter

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
import urllib.request

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

2021-12-22:06:21:51,966 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2021-12-22:06:21:51,997 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


SDK version: 1.36.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset is external and comes from the book "Rejection of Emerging Organic Contaminants by Nanofiltration and Reverse Osmosis Membranes: Effects of Fouling, Modelling and Water Reuse" [data link](http://desalination-delft.nl/wp-content/uploads/2018/06/Yangali-2010-Yangali.pdf). In water treatment, some special membranes are used to remove micropollutants (MP). MPs are ubiquitous
contaminants present in the environment (soil, water) in concentrations of µg/L or ng/L. MPs can be pharmaceutical products,
endocrine disruptor compounds, pesticides, herbicides, and other man-made organic contaminants.
The dataset contains data about MPs and their removal rates (rejection) by different types of membranes. The data also contains some parameters
used to characterise membranes of the type nanofiltration and reverse osmosis.

TODO: Get data. In the cells below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'micropol-rej'
project_folder = './micropol-project'
experiment=Experiment(ws, experiment_name)


2021-12-22:06:25:34,961 INFO     [workspace.py:290] Found the config file in: /config.json


In the next two cells, data is uploaded to the datastore of the workspace. Then the data is registered as a dataset that can be called for further use.

In [6]:
datastore = ws.get_default_datastore()

user_path = os.path.abspath(os.path.join('..'))
if user_path not in sys.path:
    sys.path.append(user_path + '/odl_user_167740/')
datastore.upload(src_dir='./data',
                 target_path='datasets/mpNFRO',
                 overwrite=True)


2021-12-22:06:24:29,907 INFO     [datastore_client.py:908] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7efd4a74a748>
2021-12-22:06:24:30,377 INFO     [azure_storage_datastore.py:754] Called AzureBlobDatastore.upload
2021-12-22:06:24:30,403 INFO     [azure_storage_datastore.py:352] Uploading an estimated of 3 files
2021-12-22:06:24:30,655 INFO     [azure_storage_datastore.py:352] Uploading ./data/.amlignore
2021-12-22:06:24:30,656 INFO     [azure_storage_datastore.py:352] Uploaded ./data/.amlignore, 1 files out of an estimated total of 3
2021-12-22:06:24:30,657 INFO     [azure_storage_datastore.py:352] Uploading ./data/.amlignore.amltmp
2021-12-22:06:24:30,657 INFO     [azure_storage_datastore.py:352] Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 3
2021-12-22:06:24:30,658 INFO     [azure_storage_datastore.py:352] Uploading ./data/micropol-removal.csv
2021-12-22:06:24:30,658 INFO     [azure_storage_datastore.py:352] Uploaded ./data/mi

Uploading an estimated of 3 files
Uploading ./data/.amlignore
Uploaded ./data/.amlignore, 1 files out of an estimated total of 3
Uploading ./data/.amlignore.amltmp
Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading ./data/micropol-removal.csv
Uploaded ./data/micropol-removal.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_35c053976a294a8a92030b00f04c7116

In [4]:
# The dataset is loaded from the Workspace. Otherwise, it is created in the workspace from the csv file uploaded in the previous cell
# NOTE: update the key to match the dataset name
found = False
key = "micro-rej"
description_text = "Rejection of micropollutants by membranes in water treatment"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        # data_path = path=(datastore, 'datasets/mpNFRO/micropol-removal.csv')
        dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'datasets/mpNFRO/micropol-removal.csv'))        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,MW,log_Kow,log_D,dipole,MV,length,width,depth,eq_width,MWCO,SR,rejection
count,161.00,161.00,161.00,161.00,161.00,161.00,161.00,161.00,161.00,161.00,161.00,161.00
mean,185.27,1.69,1.53,2.38,142.78,1.06,0.75,0.54,0.63,199.84,0.98,68.95
std,72.53,1.86,2.03,1.73,55.15,0.33,0.15,0.11,0.11,60.76,0.01,27.97
min,30.03,-3.24,-3.24,-1.68,43.50,0.47,0.43,0.34,0.38,100.00,0.96,1.00
25%,154.00,0.35,-0.27,1.02,99.40,0.78,0.64,0.45,0.54,175.00,0.97,52.00
50%,194.19,1.97,1.68,2.23,151.90,1.12,0.78,0.56,0.66,200.00,0.98,78.80
75%,233.10,3.06,3.01,3.71,186.50,1.27,0.86,0.59,0.73,200.00,0.98,90.80
max,332.00,5.71,5.88,6.30,236.20,1.79,1.00,0.75,0.81,300.00,1.00,98.60


In [5]:
# Create compute cluster

computer_cluster = "cpucluster"

computer_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                        min_nodes=0, 
                                                        max_nodes=2)
cluster_name = ComputeTarget.create(ws, computer_cluster, computer_config)

cluster_name.wait_for_completion(show_output=True)

InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Setting of the automl were chosen based on the task. The data contains non-categorical and categorical data, so featurization is specified as true. Featurization will also normalize the data. 
Since the target (rejection) is a real, we want to evaluate the accuracy of the model with the coefficient of determination R². The data is small, so 15 minutes as experiment timeout is good enough.
The number of cross validations as 5 is also reasonable.
The option to get compatible ONNX models is set to True for later saving models in onnx format.

In [10]:
# TODO: Put your automl settings here

automl_settings = {    
    "experiment_timeout_hours": 0.25,
    "enable_early_stopping": True,
    "primary_metric": 'r2_score',
    "featurization": 'auto',
    "n_cross_validations": 5
}

# TODO: Put your automl config here

automl_config = AutoMLConfig(compute_target="cpucluster",
                             task = "regression",
                             training_data=dataset,
                             label_column_name="rejection",  
                             path = project_folder,
                             debug_log = "automl_errors.log",
                             enable_onnx_compatible_models = True,
                             **automl_settings
                            )



In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster


Experiment,Id,Type,Status,Details Page,Docs Page
micropol-rej,AutoML_35da29c5-8b10-42ba-8f8b-e7e49eaccd8a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature h

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

The regressors LightGBM and XGBoostRegressor did a quite decent job with R² 0.87 and 0.89, respectively. This models did better because the data normalization with MaxAbsScaler helped in delivering the right values to the models. It is also expected that VotingEnsemble and StackEnsemble improved to a certain degree the quality of predictions, since those models are the result of agregating the results of different models.

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
remote_run.get_best_child()


Experiment,Id,Type,Status,Details Page,Docs Page
micropol-rej,AutoML_35da29c5-8b10-42ba-8f8b-e7e49eaccd8a_16,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: micropol-rej,
Id: AutoML_35da29c5-8b10-42ba-8f8b-e7e49eaccd8a_16,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='regression', working_dir='/mnt/batch/ta...
), random_state=None))], verbose=False)), ('11', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('elasticnet', ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.9478947368421053, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection='cyclic', tol=0.0001, warm_start=False))], verbose=False))], weights=[0.06666666666666667, 0.0666666666666666

In [15]:
# Get the model as onnx
best_run2, onnx_mdl = remote_run.get_output(return_onnx_model=True)

In [17]:
#TODO: Save the best model in ONNX format
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

In [18]:
#TODO: Save and register the best model in ML Azure workspace
best_run.register_model(model_name = 'mp_rejection_model', model_path = 'outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-167740', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-167740'), name=mp_rejection_model, id=mp_rejection_model:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service. Enable Application Insights is set to True.

Models can be deployed locally or in the cloud. An Azure Container Instance (ACI) web service will be used for deploying the AzureML trained best model. The model is already registered in the workspace, which means the model itself is available. But still, a few other requirements are needed:
- An entry script 'score.py'. The script has instructions for using the model, e.g. make a prediction with the data, and return a response.
- An environment '.yml' file that describes dependencies required by the model and the entry script.
- The deployment configuration, which will point into a target compute configuration (e.g. CPU cores, memory) needed to run the web service.

When deploying, the environmnet will be registered, an image will be created and the deployment will be computed. 
Once the model has been deployed the ACI service creation operation is finished. An endpoint is available now in the Workspace. The status and 
needed API key instruction will be provided in the endpoint to make use of the model.

In [19]:
sou_path = user_path + '/odl_user_167740/'
inference_config = InferenceConfig(runtime = 'python', entry_script='./score.py', conda_file='./conda_env.yml', source_directory=sou_path)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True
)


In [20]:

model = ws.models['mp_rejection_model']
service = Model.deploy(workspace=ws,
                       name = 'mem-rejection-mp',
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-22 06:55:47+00:00 Creating Container Registry if not exists..
2021-12-22 07:05:48+00:00 Registering the environment.
2021-12-22 07:05:49+00:00 Building image..
2021-12-22 07:17:03+00:00 Generating deployment configuration.
2021-12-22 07:17:06+00:00 Submitting deployment to compute..
2021-12-22 07:17:09+00:00 Checking the status of deployment mem-rejection-mp..
2021-12-22 07:19:42+00:00 Checking the status of inference endpoint mem-rejection-mp.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [37]:
# Request data goes here
data = {
    "data":
    [
        {
            "Name": 'Caffeine',
            "ID": 'CFN',
            "MW": 194.19,
            "log_Kow": -0.07, 
            "log_D": -0.45, 
            "dipole": 3.71, 
            "MV": 133.3, 
            "length": 0.98, 
            "width": 0.87, 
            "depth": 0.56, 
            "eq_width": 0.70, 
            "membrane": 'ESPA', 
            "MWCO": 200, 
            "SR": 0.99,
        },
        {
            "Name": 'Bisphenol A',
            "ID": 'BPA',
            "MW": 228.29,
            "log_Kow": 3.32, 
            "log_D": 3.86, 
            "dipole": 2.13, 
            "MV": 199.5, 
            "length": 1.25, 
            "width": 0.83, 
            "depth": 0.75, 
            "eq_width": 0.79, 
            "membrane": 'De-HL', 
            "MWCO": 175, 
            "SR": 0.97,
        },
    ],
}

body = str.encode(json.dumps(data))

url = 'http://4f1f2228-3623-479b-9189-7e499c0947f1.southcentralus.azurecontainer.io/score'
api_key = 'LYvQjhY86Wv3DD1HB0ELdag2P61uqNHS' 
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    result = result.decode('utf8').replace("'", '"')
    predict = json.loads(result)
    print(predict)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))



{"result": [87.36935990635946, 92.3259529467748]}


TODO: In the cell below, print the logs of the web service and delete the service

In [38]:
print(service.get_logs())

2021-12-22T07:19:33,069843700+00:00 - gunicorn/run 
2021-12-22T07:19:33,072621700+00:00 - iot-server/run 
2021-12-22T07:19:33,071486400+00:00 - rsyslog/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-12-22T07:19:33,113345900+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_bec455c772c2d018a0dc9c15319d853a/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-12-22T07:19:33,496979700+00:00 - iot-server/finish 1 0
2021-12-22T07:19:33,499806600+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (72)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-12-22 07:19:36,059 | root | INFO | Starting up app insights client
logging socket was

In [47]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
